```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 题目

#h(2em) 一个医药公司的新药研究部门为了掌握一种新止痛剂的疗效，设计了一个药物实验，给 24 名患有同种病痛的病人使用这种新止痛剂的以下 4 个剂量中的某一个：2，5，7，10（g），并记录每个病人病痛明显减轻的时间（min）。为了解新药的疗效与病人性别和血压有什么关系，试验过程中研究人员把病人按性别及血压的低、中、高 3 档平均分配来进行测试。通过比较每个病人血压的历史数据，从低到高分成 3 组，分别记作 0.25，0.50 和 0.75。实验结束后，公司的记录结果见表 13.32（性别以 0 表示女，1 表示男）。

#image("Chapter10_3_1.png")

#image("Chapter10_3_2.png")

#h(2em) 请你为公司建立一个模型，根据病人用药的剂量、性别和血压组别，预测出服药后病痛明显减轻的时间。

= 思路

#h(2em) 使用 statsmodels 库中的线性回归模型进行建模。

#h(2em) 下面是代码实现：
```


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
# 导入数据
data = np.loadtxt('Chapter10_3_1.csv', delimiter=',', skiprows=1)

idx = data[:, 0]  # 病人编号
time = data[:, 1]  # 病痛减轻时间
dosage = data[:, 2]  # 用药剂量
gender = data[:, 3]  # 性别
category = data[:, 4]  # 血压组别

# 数据可视化
plt.plot(dosage, time, '.')
plt.xlabel('Dosage')
plt.ylabel('Time')
plt.title('Dosage - Time')
plt.show()

plt.plot(gender, time, '.')
plt.xlabel('Gender')
plt.ylabel('Time')
plt.title('Gender - Time')
plt.show()

plt.plot(category, time, '.')
plt.xlabel('Category')
plt.ylabel('Time')
plt.title('Category - Time')
plt.show()

```{=typst}
#h(2em) 根据常识，我们知道病人的病痛减轻时间与用药量有直接的关系，除此之外，一般的话，病人病痛减轻时间与性别，血压也有明显的关系。我们假设病痛减轻时间为 $y$，用药剂量为 $x_1$，性别为 $x_2$，血压组别为 $x_3$。

#h(2em) 根据散点图，我们可以建立如下的线性回归模型：

$
  y = beta_0 + beta_1 x_1 + beta_2 x_2 + beta_3 x_3 + sum_(i j)(beta_(i j) x_i x_j) + epsilon
$
```

In [ ]:
# 由于 gender 和 category 是分类变量，不引入二次项
x_cross = [gender * category, dosage * gender, dosage * category, dosage**2]
X = np.c_[np.ones(data.shape[0]), dosage, gender, category, *x_cross]

model1 = sm.OLS(time, X).fit()

print(model1.summary())

注意到 gender 和 gender * category 的系数明显越过零点，将其舍去，重新建模。

In [ ]:
x_cross_n = [dosage * gender, dosage * category, dosage**2]
Xn = np.c_[np.ones(data.shape[0]), dosage, category, *x_cross_n]

model1n = sm.OLS(time, Xn).fit()

print(model1n.summary())

```{=typst}
$
  "time" = 43.3742 - 6.9834 "dosage" + 43.6765 "category" + 0.9551 "dosage" "gender" - 7.5294 "dosage" "category" + 0.5111 "dosage"^2
$

time: 病痛减轻的时间

dosage: 用药剂量

category: 血压组别

gender: 性别

```

In [ ]:
residuals = model1n.resid

std_resid = residuals / np.std(residuals)

for index, item in enumerate(std_resid):
  if abs(item) > 2:
    print(f'异常值第{index}号:', item)
    pass
  pass

plt.plot(idx, std_resid, '.', label='std_resid')
plt.legend()
plt.show()

存在异常值第 22 号样本，将其舍去，重新建模。

In [ ]:
Xm = np.delete(Xn, 22, axis=0)
time_m = np.delete(time, 22, axis=0)

model2 = sm.OLS(time_m, Xm).fit()

print(model2.summary())

```{=typst}
$
  "time" = 42.5896 - 6.4070 "dosage" + 43.6765 "category" + 0.7637 "dosage" "gender" - 7.5294 "dosage" "category" + 0.4544 "dosage"^2
$

time: 病痛减轻的时间

dosage: 用药剂量

category: 血压组别

gender: 性别

```